# Adding Dependeces

In [55]:
using DelimitedFiles
using ProgressMeter
using NearestNeighbors
include("../../src/forces.jl")
include("../../src/Neighbor.jl")

knn_cu (generic function with 1 method)

# Initial Conditions

In [56]:
# Running fusion of the two aggregates
# Physical Conditions
R_agg = 10

# Time model Conditions
t_f = 10000
dt = 0.1

# Neighbor Conditions
n_knn = 50
nn = 16

# Forces Conditions
r_max = 3.5
s = 1.9
K = 0.5

0.5

# Init Data

In [57]:
# Initial Data
X = Float32.(readdlm("../../data/init/Sphere/$R_agg.xyz")[3:end,2:end])

X_f = zeros(size(X,1))
for i in 1:size(X_f,1)
    A = sqrt(X[i,1]^2+X[i,2]^2+X[i,3]^2)
    if A > 0.8*R_agg
        X_f[i] = 1
    else
        X_f[i] = 2
    end
end
X_f_2 = Int.(vcat(X_f,X_f));

# Fusioning tho Spheres

In [58]:
Size = Float32((findmax(X[:,1])[1] - findmin(X[:,1])[1])/2 + 1)
X_2 = vcat(X,X)
X_2[1:size(X,1),1] = X_2[1:size(X,1),1] .- (Size)
X_2[size(X,1):end,1] = X_2[size(X,1):end,1] .+ Size

open("Test_Initial.xyz"; write=true) do f
    write(f, "$(size(X_2, 1))\n")
    write(f, "Initial Data ($(R_agg))\n")
    writedlm(f,hcat(X_f_2, X_2), ' ')
end

X = X_2;

# Forces

In [59]:
# Calculating position of every cell on the fusion
p = Progress(Int(t_f/dt),barlen=25)
for i in 0:Int(t_f/dt)

    if mod(i, n_knn) == 0
        # Calculating kNN
        kdtree = KDTree(X')
        idxs = Int.(zeros(14,size(X,1)))
        for i in 1:size(X)[1]
            # Scan neighbours
            idxs[:,i], _ = knn(kdtree, X[i,1:3], 14, true)
            # println(knn(kdtree, X[i,1:3], 14, true)[1])
            # println(idxs[:,i],)
        end
        idxs = Matrix(idxs)
    end

    # Calculating Forces
    dX = force(X, idxs, r_max, s, K)

    global X = X + dX*dt
    next!(p)
end

Progress:   1%|▏                        |  ETA: 0:35:31m

LoadError: InterruptException:

# Saving Data

In [ ]:
    # Saving final Data
    open("Test_Final[CPU].xyz"; write=true) do f
        write(f, "$(size(X, 1))\n")
        write(f, "Initial Data ($(R_agg))\n")
        writedlm(f,hcat(X_f_2, X), ' ')
    end